In [3]:
# --- Install dependencies (Colab) ---
!pip install -q torch torchvision torchaudio scikit-learn matplotlib

In [4]:
# --- Imports ---
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from google.colab import drive
from sklearn.metrics import mean_squared_error
import numpy as np
import math
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

# --- Mount Google Drive ---
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# --- Positional Encoding ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1200):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

In [6]:
# --- Patch Embedding via Conv ---
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels=1, emb_size=128):
        super().__init__()
        self.proj = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, stride=2, padding=1),
            nn.GELU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.GELU(),
            nn.Conv2d(64, emb_size, kernel_size=3, stride=2, padding=1),
            nn.GELU()
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # (B, 1, T, C)
        x = self.proj(x)    # (B, emb_size, H, W)
        B, E, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)  # (B, N_patches, emb_size)
        return x

In [7]:
# --- Transformer Encoder Block ---
class TransformerBlock(nn.Module):
    def __init__(self, emb_size=256, heads=4, ff_dim=512, dropout=0.2):
        super().__init__()
        self.attn = nn.MultiheadAttention(emb_size, heads, dropout=dropout, batch_first=True)
        self.ff = nn.Sequential(
            nn.Linear(emb_size, ff_dim),
            nn.GELU(),
            nn.Linear(ff_dim, emb_size)
        )
        self.norm1 = nn.LayerNorm(emb_size)
        self.norm2 = nn.LayerNorm(emb_size)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        attn_out, _ = self.attn(x, x, x)
        x = self.norm1(x + self.drop(attn_out))
        ff_out = self.ff(x)
        x = self.norm2(x + self.drop(ff_out))
        return x

In [8]:
# --- Full Model ---
class EEGTransformerRegressor(nn.Module):
    def __init__(self, emb_size=256, num_layers=4):
        super().__init__()
        self.patch_embed = PatchEmbedding(1, emb_size)
        self.pos_encoder = PositionalEncoding(emb_size)
        self.transformer = nn.Sequential(*[
            TransformerBlock(emb_size) for _ in range(num_layers)
        ])
        self.regressor = nn.Sequential(
            nn.LayerNorm(emb_size, eps=1e-5),
            nn.Linear(emb_size, 64),
            nn.GELU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        x = self.patch_embed(x)
        x = self.pos_encoder(x)
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.regressor(x)

In [9]:
class EEGNPZDataset(Dataset):
    def __init__(self, file_paths, use_fraction=0.75):
        self.samples = []
        for path in file_paths:
            data = np.load(path)
            eeg = data['EEG']
            labels = data['labels'][:, -2:]

            N = int(len(eeg) * use_fraction)
            self.samples.extend(zip(eeg[:N], labels[:N]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x, y = self.samples[idx]
        if np.isnan(x).any() or np.isnan(y).any():
            x = np.nan_to_num(x, nan=0.0)
            y = np.nan_to_num(y, nan=0.0)
        x = (x - np.mean(x)) / (np.std(x) + 1e-6)
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [10]:
def find_nan_outputs(model, file_paths, device):
    model.eval()
    model.to(device)

    bad_samples = []  # (file_path, sample_index)

    for path in file_paths:
        print(f"Checking file: {path}")
        data = np.load(path)
        eeg = data['EEG']
        for i in range(len(eeg)):
            x = eeg[i]
            x = (x - np.mean(x)) / (np.std(x) + 1e-6)  # normalize
            x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(0).to(device)  # shape: (1, 500, 129)

            try:
                with torch.no_grad(), torch.amp.autocast('cuda'):
                    output = model(x_tensor)
                if torch.isnan(output).any():
                    print(f"NaN in output from file {path} index {i}")
                    bad_samples.append((path, i))
            except Exception as e:
                print(f"Error on file {path} index {i}: {e}")
                bad_samples.append((path, i))

    return bad_samples

In [11]:
if False:
    model = EEGTransformerRegressor()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    DATA_DIR = "/content/drive/MyDrive/data"
    FILES = [
        "Direction_task_with_dots_synchronised_max.npz",
        "Direction_task_with_dots_synchronised_min.npz",
        "Direction_task_with_processing_speed_synchronised_max.npz",
        "Direction_task_with_processing_speed_synchronised_min.npz",
        #"Position_task_with_dots_synchronised_max.npz",
        #"Position_task_with_dots_synchronised_min.npz"
    ]
    file_paths = [os.path.join(DATA_DIR, f) for f in FILES]

    bad_samples = find_nan_outputs(model, file_paths, device)
    print(f"Found {len(bad_samples)} samples with NaN outputs.")

In [12]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, save_dir, patience=5):
    scaler = torch.amp.GradScaler('cuda')
    best_rmse = float('inf')
    train_losses = []
    val_losses = []
    val_rmses = []
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()

            with torch.amp.autocast('cuda'):
                outputs = model(inputs)
                loss = criterion(outputs, targets)

            if not torch.isfinite(loss).all():
                print("⚠️ Skipping batch with non-finite loss")
                continue

            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * inputs.size(0)

        avg_train_loss = running_loss / len(train_loader.dataset)
        train_losses.append(avg_train_loss)

        model.eval()
        preds, gts = [], []
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                preds.append(outputs.cpu().numpy())
                gts.append(targets.cpu().numpy())

        preds = np.concatenate(preds)
        gts = np.concatenate(gts)
        loss = mean_squared_error(gts, preds)
        rmse = np.sqrt(loss)
        val_losses.append(loss)
        val_rmses.append(rmse)

        scheduler.step()

        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val RMSE: {rmse:.4f}")

        if rmse < best_rmse:
            best_rmse = rmse
            patience_counter = 0
            torch.save(model.state_dict(), os.path.join(save_dir, "best_model.pt"))
            print("✅ Saved new best model.")
        else:
            patience_counter += 1
            print(f"⏳ No improvement. Patience: {patience_counter}/{patience}")
            if patience_counter >= patience:
                print("🛑 Early stopping triggered.")
                break

    # Save loss plots
    plt.figure()
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss / RMSE')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.savefig(os.path.join(save_dir, 'loss_plot.png'))
    plt.close()

In [13]:
DATA_DIR = "/content/drive/MyDrive/data"
FILES = [
    "Direction_task_with_dots_synchronised_max.npz",
    "Direction_task_with_dots_synchronised_min.npz",
    "Direction_task_with_processing_speed_synchronised_max.npz",
    "Direction_task_with_processing_speed_synchronised_min.npz",
    #"Position_task_with_dots_synchronised_max.npz",
    #"Position_task_with_dots_synchronised_min.npz"
]
file_paths = [os.path.join(DATA_DIR, f) for f in FILES]

dataset = EEGNPZDataset(file_paths)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True, persistent_workers=True)

In [14]:
# --- Run Everything ---
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = EEGTransformerRegressor().to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=2e-5)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5)

    # --- Create save directory ---
    base_save_dir = "/content/drive/MyDrive/cnn_transformer"
    os.makedirs(base_save_dir, exist_ok=True)
    num_try = 0
    for folder in sorted(os.listdir(base_save_dir)):
        full_path = os.path.join(base_save_dir, folder)
        if os.path.isdir(full_path) and folder.startswith("Try") and any(os.listdir(full_path)):
            num_try += 1

    save_dir = os.path.join(base_save_dir, f"Try{num_try + 1}")
    os.makedirs(save_dir, exist_ok=True)

    train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, device=device, save_dir=save_dir)

Epoch 1: 100%|██████████| 463/463 [02:26<00:00,  3.15it/s]


Epoch 1 | Train Loss: 41077.4176 | Val RMSE: 201.9904
✅ Saved new best model.


Epoch 2: 100%|██████████| 463/463 [02:23<00:00,  3.23it/s]


Epoch 2 | Train Loss: 38701.6215 | Val RMSE: 194.6881
✅ Saved new best model.


Epoch 3: 100%|██████████| 463/463 [02:23<00:00,  3.23it/s]


...

Epoch 93 | Train Loss: 4478.2865 | Val RMSE: 74.8270
⏳ No improvement. Patience: 3/5


Epoch 94: 100%|██████████| 463/463 [02:23<00:00,  3.23it/s]


Epoch 94 | Train Loss: 4324.8250 | Val RMSE: 74.4577
⏳ No improvement. Patience: 4/5


Epoch 95: 100%|██████████| 463/463 [02:23<00:00,  3.23it/s]


Epoch 95 | Train Loss: 4207.6132 | Val RMSE: 74.6333
⏳ No improvement. Patience: 5/5
🛑 Early stopping triggered.
